In [11]:
import torch.nn as nn
from torchsummary import summary

In [65]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.mod = nn.Sequential(
            nn.Conv2d(1, 64, 3),
            nn.ReLU(True),
            nn.Conv2d(64,64, 3),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64,128, 3),
            nn.ReLU(True),
            nn.Conv2d(128,128, 3),
            nn.ReLU(True)
        )
       #self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        y = self.mod(x) 
        return (y)
        # x = F.relu(self.conv1(x))
        # return F.relu(self.conv2(x))

In [9]:
class UNet(nn.Module):
    def __init__(self,IN_CHANNELS=1,OUT_CHANNELS=1,NUM_FILTERS=64,NUM_LAYERS=4):
        super().__init__()
        self.conv1 = nn.Conv2d()
        # self.conv2 = nn.Conv2d(20, 20, 5)
        # self.max_pool = nn.MaxPool(kerner_size=2, stride=2)

In [66]:
model = Model()

In [57]:
import torch

In [67]:
summary(model, input_size = (1, 572, 572))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 570, 570]             640
              ReLU-2         [-1, 64, 570, 570]               0
            Conv2d-3         [-1, 64, 568, 568]          36,928
              ReLU-4         [-1, 64, 568, 568]               0
            Conv2d-5        [-1, 128, 566, 566]          73,856
              ReLU-6        [-1, 128, 566, 566]               0
            Conv2d-7        [-1, 128, 564, 564]         147,584
              ReLU-8        [-1, 128, 564, 564]               0
Total params: 259,008
Trainable params: 259,008
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.25
Forward/backward pass size (MB): 1879.32
Params size (MB): 0.99
Estimated Total Size (MB): 1881.56
----------------------------------------------------------------
